# Pain in the Net
Replication of *Deeper Image Quality Transfer: Training Low-Memory Neural Networks for 3D Images*


Code by:

Tyler Spears - tas6hh@virginia.edu

Dr. Tom Fletcher

---

Source work:
`S. B. Blumberg, R. Tanno, I. Kokkinos, and D. C. Alexander, “Deeper Image Quality Transfer: Training Low-Memory Neural Networks for 3D Images,” in Medical Image Computing and Computer Assisted Intervention – MICCAI 2018, Cham, 2018, pp. 118–125, doi: 10.1007/978-3-030-00928-1_14.`


# Imports & Environment Setup

## Imports

In [1]:
# Automatically re-import project-specific modules.
%load_ext autoreload
%autoreload 1

# imports
import collections
import dataclasses
from dataclasses import dataclass
import functools
import io
import datetime
import time

import math
import itertools
import os
import shutil
import pathlib
import copy
import pdb
import inspect
import IPython
import random
import subprocess
import sys
import warnings
from pathlib import Path
import typing
from typing import Generator

import ants
import dipy
import dipy.core
import dipy.reconst
import dipy.reconst.dti
import dipy.segment.mask
import dotenv

# visualization libraries
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Try importing GPUtil for printing GPU specs.
# May not be installed if using CPU only.
try:
    import GPUtil
except ImportError:
    warnings.warn("WARNING: Package GPUtil not found, cannot print GPU specs")
from tabulate import tabulate
from IPython.display import display, Markdown

# Data management libraries.
import nibabel as nib
import nilearn
import nilearn.plotting
import natsort
from natsort import natsorted

# Computation & ML libraries.
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import torchvision
import torchio
import pytorch_lightning as pl

import skimage
import skimage.feature
import skimage.filters
import skimage.measure
import scipy
import einops
import einops.layers
import einops.layers.torch

plt.rcParams.update({"figure.autolayout": True})
plt.rcParams.update({"figure.facecolor": [1.0, 1.0, 1.0, 1.0]})

# Set print options for ndarrays/tensors.
np.set_printoptions(suppress=True, threshold=100, linewidth=88)
torch.set_printoptions(sci_mode=False, threshold=100, linewidth=88)

/opt/miniconda/envs/pitn/lib/python3.8/site-packages/nilearn/datasets/__init__.py:87: FutureWarning:

Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.



In [2]:
# Update notebook's environment variables with direnv.
# This requires the python-dotenv package, and direnv be installed on the system
# This will not work on Windows.
# NOTE: This is kind of hacky, and not necessarily safe. Be careful...
# Libraries needed on the python side:
# - os
# - subprocess
# - io
# - dotenv

# Form command to be run in direnv's context. This command will print out
# all environment variables defined in the subprocess/sub-shell.
command = "direnv exec {} /usr/bin/env".format(os.getcwd())
# Run command in a new subprocess.
proc = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True, cwd=os.getcwd())
# Store and format the subprocess' output.
proc_out = proc.communicate()[0].strip().decode("utf-8")
# Use python-dotenv to load the environment variables by using the output of
# 'direnv exec ...' as a 'dummy' .env file.
dotenv.load_dotenv(stream=io.StringIO(proc_out), override=True);

In [3]:
# Project-specific scripts
# It's easier to import it this way rather than make an entirely new package, due to
# conflicts with local packages and anaconda installations.
# You made me do this, poor python package management!!
if "PROJECT_ROOT" in os.environ:
    lib_location = str(Path(os.environ["PROJECT_ROOT"]).resolve())
else:
    lib_location = str(Path("../../").resolve())
if lib_location not in sys.path:
    sys.path.insert(0, lib_location)
import lib as pitn

# Include the top-level lib module along with its submodules.
%aimport lib
# Grab all submodules of lib, not including modules outside of the package.
includes = list(
    filter(
        lambda m: m.startswith("lib."),
        map(lambda x: x[1].__name__, inspect.getmembers(pitn, inspect.ismodule)),
    )
)
# Run aimport magic with constructed includes.
ipy = IPython.get_ipython()
ipy.run_line_magic("aimport", ", ".join(includes))

In [4]:
# torch setup
# allow for CUDA usage, if available
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
# keep device as the cpu
# device = torch.device('cpu')
print(device)

cuda


## Specs Recording

In [5]:
%%capture --no-stderr cap
# Capture output and save to log. Needs to be at the *very first* line of the cell.
# Watermark
%load_ext watermark
%watermark --author "Tyler Spears" --updated --iso8601  --python --machine --iversions --githash
if torch.cuda.is_available():

    # GPU information
    # Taken from
    # <https://www.thepythoncode.com/article/get-hardware-system-information-python>.
    # If GPUtil is not installed, skip this step.
    try:
        gpus = GPUtil.getGPUs()
        print("=" * 50, "GPU Specs", "=" * 50)
        list_gpus = []
        for gpu in gpus:
            # get the GPU id
            gpu_id = gpu.id
            # name of GPU
            gpu_name = gpu.name
            driver_version = gpu.driver
            cuda_version = torch.version.cuda
            # get total memory
            gpu_total_memory = f"{gpu.memoryTotal}MB"
            gpu_uuid = gpu.uuid
            list_gpus.append(
                (
                    gpu_id,
                    gpu_name,
                    driver_version,
                    cuda_version,
                    gpu_total_memory,
                    gpu_uuid,
                )
            )

        print(
            tabulate(
                list_gpus,
                headers=(
                    "id",
                    "Name",
                    "Driver Version",
                    "CUDA Version",
                    "Total Memory",
                    "uuid",
                ),
            )
        )
    except NameError:
        print("CUDA Version: ", torch.version.cuda)

else:
    print("CUDA not in use, falling back to CPU")

In [6]:
# cap is defined in an ipython magic command
print(cap)

Author: Tyler Spears

Last updated: 2021-05-20T02:46:34.798459+00:00

Python implementation: CPython
Python version       : 3.8.8
IPython version      : 7.22.0

Compiler    : GCC 7.3.0
OS          : Linux
Release     : 5.4.0-72-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 8
Architecture: 64bit

Git hash: 13e47db6ff51b6e5cba2e0bcd0d0ab4bad82df84

torchvision      : 0.9.1
matplotlib       : 3.4.1
torchio          : 0.18.37
dipy             : 1.4.0
ipywidgets       : 7.6.3
scipy            : 1.5.3
pytorch_lightning: 1.2.6
nilearn          : 0.7.1
GPUtil           : 1.4.0
seaborn          : 0.11.1
ants             : 0.2.7
nibabel          : 3.2.1
torch            : 1.8.1
json             : 2.0.9
pandas           : 1.2.3
natsort          : 7.1.1
sys              : 3.8.8 (default, Feb 24 2021, 21:46:12) 
[GCC 7.3.0]
einops           : 0.3.0
IPython          : 7.22.0
skimage          : 0.18.1
numpy            : 1.20.2

================================================== GPU 

## Data Variables & Definitions Setup

In [7]:
# Set up directories
data_dir = pathlib.Path(os.environ["DATA_DIR"]) / "hcp"
assert data_dir.exists()
write_data_dir = pathlib.Path(os.environ["WRITE_DATA_DIR"]) / "hcp"
assert write_data_dir.exists()
results_dir = pathlib.Path(os.environ["RESULTS_DIR"])
assert results_dir.exists()

## Experiment Logging Setup

In [8]:
# Experiment logging setup
EXPERIMENT_NAME = "debug_norm"

ts = datetime.datetime.now().replace(microsecond=0).isoformat()
# Break ISO format because many programs don't like having colons ':' in a filename.
ts = ts.replace(":", "_")
experiment_name = ts + "__" + EXPERIMENT_NAME

# Create temporary directory for results directory, in case experiment does not finish.
tmp_results_dir = results_dir / "tmp"
tmp_dirs = list(tmp_results_dir.glob("*"))

# Only keep up to N tmp results.
n_tmp_to_keep = 3
if len(tmp_dirs) > (n_tmp_to_keep - 1):
    print(f"More than {n_tmp_to_keep} temporary results, culling to the most recent")
    tmps_to_delete = natsorted([str(tmp_dir) for tmp_dir in tmp_dirs])[
        : -(n_tmp_to_keep - 1)
    ]
    for tmp_dir in tmps_to_delete:
        shutil.rmtree(tmp_dir)
        print("Deleted temporary results directory ", tmp_dir)

experiment_results_dir = tmp_results_dir / experiment_name
# Final target directory, to be made when experiment is complete.
final_experiment_results_dir = results_dir / experiment_name

(experiment_results_dir).mkdir(parents=True, exist_ok=True)
print(
    "Experiment results directory: ",
    experiment_results_dir,
)
assert experiment_results_dir.exists()

experiment_results_log = experiment_results_dir / "log.txt"
with open(experiment_results_log, "a+") as f:
    f.write(f"Experiment Name: {experiment_name}\n")
    f.write(f"Timestamp: {ts}\n")
    # cap is defined in an ipython magic command
    f.write(f"Environment and Hardware Info:\n {cap}\n\n")

Experiment results directory:  /home/jovyan/work/pitn/results/tmp/2021-05-20T02_46_43__debug_norm


# Parameters and Function Definitions

## Parameters

In [9]:
downsample_factor = 2
# Include b=0 shells and b=1000 shells for DTI fitting.
bval_range = (0, 1500)
dti_fit_method = "WLS"

### Patch Parameters

In [10]:
# Patch parameters
batch_size = 32
# 6 channels for the 6 DTI components
channels = 6

# Output patch shapes
h_out = 14
w_out = 14
d_out = 14

# This is the factor that determines how over-extended the input patch should be
# relative to the size of the full-res patch.
# $low_res_patch_dim = \frac{full_res_patch_dim}{downsample_factor} \times low_res_sample_extension$
# A value of 1 indicates that the input patch dims will be exactly divided by the
# downsample factor. A dilation > 1 increases the "spatial extent" of the input
# patch, providing information outside of the target HR patch.
low_res_sample_extension = 1.57

# Output shape after shuffling.
output_patch_shape = (channels, h_out, w_out, d_out)
output_spatial_patch_shape = output_patch_shape[1:]

# Input patch parameters
h_in = round(h_out / (downsample_factor) * low_res_sample_extension)
w_in = round(w_out / (downsample_factor) * low_res_sample_extension)
d_in = round(d_out / (downsample_factor) * low_res_sample_extension)
input_patch_shape = (channels, h_in, w_in, d_in)
input_spatial_patch_shape = input_patch_shape[1:]

# Pre-shuffle output patch sizes.
unshuffled_channels_out = channels * downsample_factor ** 3
# Output before shuffling
unshuffled_output_patch_shape = (unshuffled_channels_out, h_in, w_in, d_in)

# Patch size in FR-space when accounting for the low-res over-extension/over-sampling
# factor.
fr_extension_patch_size = tuple(
    np.asarray(input_spatial_patch_shape) * downsample_factor
)
fr_extension_amount = tuple(
    np.asarray(fr_extension_patch_size) - np.asarray(output_spatial_patch_shape)
)

In [11]:
# Data parameters.
num_subject_samples = 10
# Should the data be normalized as a pre-processing step?
data_norm_method = "channels"

### Training and Testing Setup

In [12]:
# Training/testing parameters.
test_percent = 0.2
# test_percent = 0.5
train_percent = 1 - test_percent
# val_percent = 0.1

# NN parameters.
max_epochs = 50
network_norm_method = None
train_loss_name = "MSE"

# Spline interpolation baseline parameters.
spline_interp_order = 3

In [13]:
# Number of voxels to dilate the mask in FR space.
# Dilate to allow the outer-most perimeter to be completely outside the brain, and no
# more.
dilation_size = math.ceil((max(fr_extension_patch_size) + 1) / 2)
# 12 is too much, nearly doubles the volume of the brain mask. Lower it some more...
dilation_size = dilation_size // 3
# Even 4 is a chunky bit! Just make it 0.
dilation_size = 0

In [14]:
with open(experiment_results_log, "a+") as f:
    f.write(f"Downsample Factor: {downsample_factor}\n")
    f.write(f"Low-Res oversampling factor: {low_res_sample_extension}\n")
    f.write(f"DTI Fit Method: {dti_fit_method}\n")
    f.write(f"Input Patch Size: {input_patch_shape}\n")
    f.write(f"Output Patch Size: {output_patch_shape}\n")
    f.write(f"Brain Mask Dilation Size: {dilation_size}\n")
    f.write(f"Batch Size: {batch_size}\n")

## Function Definitions

In [15]:
BoxplotStats = collections.namedtuple(
    "BoxplotStats",
    ["low_outliers", "low", "q1", "median", "q3", "high", "high_outliers"],
)


def batch_boxplot_stats(batch):
    """Quick calculation of a batch of 1D values for showing boxplot stats."""
    q1, median, q3 = np.quantile(batch, q=[0.25, 0.5, 0.75], axis=1)
    iqr = q3 - q1
    low = q1 - (1.5 * iqr)
    high = q3 + (1.5 * iqr)
    low_outliers = list()
    high_outliers = list()
    # Number of outliers may be different for each batch, so it needs to be a list of
    # arrays.
    for i_batch in range(len(batch)):
        batch_i = batch[i_batch]
        low_i = low[i_batch]
        low_outliers.append(batch_i[np.where(batch_i < low_i)])
        high_i = high[i_batch]
        high_outliers.append(batch_i[np.where(batch_i > high_i)])

    return BoxplotStats(low_outliers, low, q1, median, q3, high, high_outliers)

In [16]:
# Quick check on full volume/batch distributions.


def print_channel_dists(vols):
    if not torch.is_tensor(vols):
        t_vols = torch.as_tensor(vols)
    else:
        t_vols = vols

    if t_vols.ndim == 4:
        t_vols = t_vols[None, ...]

    results = "means | vars\n"
    for t_vol_i in t_vols:
        mean_i = torch.mean(t_vol_i, dim=(1, 2, 3))
        var_i = torch.var(t_vol_i, dim=(1, 2, 3))
        mvs = [
            (f"{mv[0]} | {mv[1]}\n")
            for mv in torch.stack([mean_i, var_i], dim=-1).tolist()
        ]
        results = results + "".join(mvs)
        results = results + ("=" * (len(mvs[-1]) - 1)) + "\n"

    print(results)

# Data Loading

## Subject ID Selection

In [17]:
# Find data directories for each subject.
subj_dirs: dict = dict()

selected_ids = [
    "397154",
    "224022",
    "140117",
    "751348",
    "894774",
    "156637",
    "227432",
    "303624",
    "185947",
    "810439",
    "753251",
    "644246",
    "141422",
    "135528",
    "103010",
    "700634",
]

## Sub-set the chosen participants for dev and debugging!
selected_ids = random.sample(selected_ids, num_subject_samples)
warnings.warn(
    "WARNING: Sub-selecting participants for dev and debugging. "
    + f"Subj IDs selected: {selected_ids}"
)
# ### A nested warning! For debugging only.
# warnings.warn("WARNING: Mixing training and testing subjects")
# selected_ids.append(selected_ids[0])
# ###
##

selected_ids = natsorted(list(map(lambda s: int(s), selected_ids)))

for subj_id in selected_ids:
    subj_dirs[subj_id] = data_dir / f"{subj_id}/T1w/Diffusion"
    assert subj_dirs[subj_id].exists()
subj_dirs

<ipython-input-17-7175a7d586cd>:25: UserWarning:




{135528: PosixPath('/mnt/storage/data/pitn/hcp/135528/T1w/Diffusion'),
 141422: PosixPath('/mnt/storage/data/pitn/hcp/141422/T1w/Diffusion'),
 156637: PosixPath('/mnt/storage/data/pitn/hcp/156637/T1w/Diffusion'),
 227432: PosixPath('/mnt/storage/data/pitn/hcp/227432/T1w/Diffusion'),
 303624: PosixPath('/mnt/storage/data/pitn/hcp/303624/T1w/Diffusion'),
 644246: PosixPath('/mnt/storage/data/pitn/hcp/644246/T1w/Diffusion'),
 700634: PosixPath('/mnt/storage/data/pitn/hcp/700634/T1w/Diffusion'),
 751348: PosixPath('/mnt/storage/data/pitn/hcp/751348/T1w/Diffusion'),
 753251: PosixPath('/mnt/storage/data/pitn/hcp/753251/T1w/Diffusion'),
 810439: PosixPath('/mnt/storage/data/pitn/hcp/810439/T1w/Diffusion')}

The 90 scans are taken from the $b=1000 \ s/mm^2$. However, the $b=0$ shells are still required for fitting the diffusion tensors (DTI's), so those will need to be kept, too.

To find those, sub-select with the $0 < bvals < 1500$, or roughly thereabout. A b-val of $995$ or $1005$ still counts as a b=1000.

In [18]:
with open(experiment_results_log, "a+") as f:
    f.write(f"Selected Subjects: {selected_ids}\n")

## Loading and Preprocessing

In [19]:
# Set up the transformation pipeline.
# Pad the original image & maks to ensure that lr patches cannot have invalid indices.
# fr_spatial_padding = tuple(
#     (np.ceil(np.asarray(input_spatial_patch_shape) * downsample_factor / 2) + 1).astype(
#         int
#     )
# )

preproc_transforms = torchio.Compose(
    [
        torchio.transforms.ToCanonical(include=("dwi", "brain_mask"), copy=False),
        pitn.transforms.BValSelectionTransform(
            bval_range=bval_range,
            bval_key="bvals",
            bvec_key="bvecs",
            include="dwi",
            copy=False,
        ),
        # Pad by the dilation factor, then dilate the mask.
        torchio.transforms.Pad(
            dilation_size,
            padding_mode=0,
            include=("dwi", "brain_mask"),
            copy=False,
        ),
        pitn.transforms.DilateMaskTransform(
            dilation_size=dilation_size, include=("brain_mask",), copy=False
        ),
        # Pad by the amount of extension voxels in FR space, so LR indices cannot
        # go out of bounds.
        torchio.transforms.Pad(
            fr_extension_amount,
            padding_mode=0,
            include=("dwi", "brain_mask"),
            copy=False,
        ),
        # Ensure FR dims are divisible by the downsample factor, to more reliably
        # convert between FR indices and LR indices.
        torchio.transforms.EnsureShapeMultiple(
            downsample_factor, method="pad", include=("dwi", "brain_mask"), copy=False
        ),
        pitn.transforms.MeanDownsampleTransform(
            downsample_factor,
            include=("dwi", "brain_mask"),
            keep={"dwi": "fr_dwi", "brain_mask": "fr_brain_mask"},
            copy=False,
        ),
        pitn.transforms.RenameImageTransform(
            {"dwi": "lr_dwi", "brain_mask": "lr_brain_mask"}, copy=False
        ),
        pitn.transforms.FitDTITransform(
            "bvals",
            "bvecs",
            "fr_brain_mask",
            fit_method=dti_fit_method,
            include=("fr_dwi"),
            copy=False,
        ),
        pitn.transforms.FitDTITransform(
            "bvals",
            "bvecs",
            "lr_brain_mask",
            fit_method=dti_fit_method,
            include=("lr_dwi"),
            copy=False,
        ),
        pitn.transforms.RenameImageTransform(
            {"fr_dwi": "fr_dti", "lr_dwi": "lr_dti"}, copy=False
        ),
        pitn.transforms.ImageToDictTransform(
            include=("lr_dti", "lr_brain_mask"), copy=False
        ),
    ]
)

In [ ]:
# Import all image data into a sequence of `torchio.Subject` objects.
subj_data: dict = dict()
summary_stats_fmt = "github"
summary_stats_header = [
    "Subj ID",
    "Resolution",
    "Channel Index",
    "Num Outliers (Lower)",
    "Low",
    "25th Percentile",
    "Median",
    "75th Percentile",
    "High",
    "Num Outliers (Upper)",
]
dti_channel_names = ["Dxx", "Dxy", "Dyy", "Dxz", "Dyz", "Dzz"]
subj_stats = dict()
for k in summary_stats_header:
    subj_stats[k] = list()

for subj_id, subj_dir in subj_dirs.items():
    # Sub-select volumes with only bvals in a certain range. E.x. bvals <= 1100 mm/s^2,
    # a.k.a. only the b=0 and b=1000 shells.
    bvals = torch.as_tensor(np.loadtxt(subj_dir / "bvals").astype(int))
    bvecs = torch.as_tensor(np.loadtxt(subj_dir / "bvecs"))
    # Reshape to be N x 3
    if bvecs.shape[0] == 3:
        bvecs = bvecs.T

    brain_mask = torchio.LabelMap(
        subj_dir / "nodif_brain_mask.nii.gz",
        type=torchio.LABEL,
        channels_last=False,
    )
    brain_mask.set_data(brain_mask.data.bool())
    mask_volume = brain_mask["data"].sum()
    print(f"Brain mask volume before dilation: {mask_volume}")
    dwi = torchio.ScalarImage(
        subj_dir / "data.nii.gz",
        type=torchio.INTENSITY,
        bvals=bvals,
        bvecs=bvecs,
        reader=pitn.io.nifti_reader,
        channels_last=True,
    )

    subject_dict = torchio.Subject(subj_id=subj_id, dwi=dwi, brain_mask=brain_mask)
    preproced_subj = preproc_transforms(subject_dict)
    dilated_mask_volume = preproced_subj["fr_brain_mask"]["data"].sum()
    print(f"Dilated mask volume: {dilated_mask_volume}")
    print(f"Mask volume difference: {dilated_mask_volume - mask_volume}")
    # Subject-and-channel-wise standardization/normalization of both the LR and FR vols.
    # Note that LR and FR images should have the same means, but *not* the same variances.
    fr_vol = preproced_subj.fr_dti.tensor
    fr_mask = preproced_subj.fr_brain_mask.tensor.bool()
    masked_fr_vol = torch.masked_select(fr_vol, fr_mask).reshape(fr_vol.shape[0], -1)
    fr_channel_means = masked_fr_vol.mean(dim=1)
    fr_channel_means = fr_channel_means.reshape(-1, 1, 1, 1)
    fr_channel_vars = masked_fr_vol.var(dim=1)
    fr_channel_vars = fr_channel_vars.reshape(-1, 1, 1, 1)

    # Store subject-and-channel-wise means and vars in order to reverse the normalization
    # for the final visualization/output.
    preproced_subj["fr_means"] = fr_channel_means.detach().cpu().numpy()
    preproced_subj["fr_vars"] = fr_channel_vars.detach().cpu().numpy()

    lr_vol = preproced_subj.lr_dti["data"]
    lr_mask = preproced_subj.lr_brain_mask["data"].bool()
    masked_lr_vol = torch.masked_select(lr_vol, lr_mask).reshape(lr_vol.shape[0], -1)
    lr_channel_means = masked_lr_vol.mean(dim=1)
    lr_channel_means = lr_channel_means.reshape(-1, 1, 1, 1)
    lr_channel_vars = masked_lr_vol.var(dim=1)
    lr_channel_vars = lr_channel_vars.reshape(-1, 1, 1, 1)

    # Store subject-and-channel-wise means and vars in order to reverse the normalization
    # for the final visualization/output.
    preproced_subj["lr_means"] = lr_channel_means.detach().cpu().numpy()
    preproced_subj["lr_vars"] = lr_channel_vars.detach().cpu().numpy()

    # Print and log some statistics of the subject data.
    fr_vol_stats = batch_boxplot_stats(masked_fr_vol)
    lr_vol_stats = batch_boxplot_stats(masked_lr_vol)
    subj_stats["Subj ID"].extend(
        list(
            itertools.repeat(
                subj_id, len(fr_vol_stats.median) + len(lr_vol_stats.median)
            )
        )
    )
    subj_stats["Resolution"].extend(
        list(
            itertools.chain(
                itertools.repeat("Full", len(fr_vol_stats.median)),
                itertools.repeat("Low", len(lr_vol_stats.median)),
            )
        )
    )
    subj_stats["Channel Index"].extend(dti_channel_names * 2)

    subj_stats["Num Outliers (Lower)"].extend(
        list(
            itertools.chain(
                map(len, fr_vol_stats.low_outliers), map(len, lr_vol_stats.low_outliers)
            )
        )
    )
    subj_stats["Low"].extend(list(np.concatenate([fr_vol_stats.low, lr_vol_stats.low])))
    subj_stats["25th Percentile"].extend(
        list(np.concatenate([fr_vol_stats.q1, lr_vol_stats.q1]))
    )

    subj_stats["Median"].extend(
        list(np.concatenate([fr_vol_stats.median, lr_vol_stats.median]))
    )

    subj_stats["75th Percentile"].extend(
        list(np.concatenate([fr_vol_stats.q3, lr_vol_stats.q3]))
    )
    subj_stats["High"].extend(
        list(np.concatenate([fr_vol_stats.high, lr_vol_stats.high]))
    )
    subj_stats["Num Outliers (Upper)"].extend(
        list(
            itertools.chain(
                map(len, fr_vol_stats.high_outliers),
                map(len, lr_vol_stats.high_outliers),
            )
        )
    )

    if isinstance(data_norm_method, str) and "channel" in data_norm_method.casefold():
        # Standardize the volumes.
        preproced_subj.fr_dti.set_data(
            pitn.data.norm.normalize_dti(fr_vol, fr_channel_means, fr_channel_vars)
        )
        preproced_subj.lr_dti["data"] = pitn.data.norm.normalize_dti(
            lr_vol, lr_channel_means, lr_channel_vars
        )

    subj_data[subj_id] = preproced_subj
    print("=" * 20)
#     breakpoint()

print("===Data Loaded & Transformed===")

subj_stats_str = tabulate(
    subj_stats, headers=summary_stats_header, tablefmt=summary_stats_fmt
)

Brain mask volume before dilation: 635172
Loading NIFTI image: /mnt/storage/data/pitn/hcp/135528/T1w/Diffusion/data.nii.gz
	Loaded NIFTI image
Selecting with bvals: Subject 135528...Selected
Downsampling: Subject 135528...Downsampled
Fitting to DTI: Subject 135528......DWI shape: torch.Size([108, 162, 190, 162])......DTI shape: (6, 162, 190, 162)...Fitted DTI model: torch.Size([6, 162, 190, 162])
Fitting to DTI: Subject 135528......DWI shape: torch.Size([108, 81, 95, 81])......DTI shape: (6, 81, 95, 81)...Fitted DTI model: torch.Size([6, 81, 95, 81])
Dilated mask volume: 635172
Mask volume difference: 0
Brain mask volume before dilation: 729001
Loading NIFTI image: /mnt/storage/data/pitn/hcp/141422/T1w/Diffusion/data.nii.gz
	Loaded NIFTI image
Selecting with bvals: Subject 141422...Selected
Downsampling: Subject 141422...Downsampled
Fitting to DTI: Subject 141422......DWI shape: torch.Size([108, 162, 190, 162])......DTI shape: (6, 162, 190, 162)...Fitted DTI model: torch.Size([6, 162, 

In [ ]:
with open(experiment_results_log, "a+") as f:
    f.write(f"Preprocessing transformation pipeline: {str(preproc_transforms)}\n")
    f.write(f"Data Summary Statistics, no normalization:\n {subj_stats_str}\n\n")

In [ ]:
display(Markdown(subj_stats_str))

In [ ]:
subj_dataset = torchio.SubjectsDataset(list(subj_data.values()), load_getitem=False)

In [ ]:
# print_channel_dists(
#     torch.stack([subj["fr_dti"]["data"] for subj in subj_data.values()])
# )

In [ ]:
# print_channel_dists(
#     torch.stack([subj["lr_dti"]["data"] for subj in subj_data.values()])
# )

# Model Training

## Set Up Patch-Based Data Loaders

In [ ]:
# Data train/validation/test split

num_subjs = len(subj_dataset)
num_test_subjs = int(np.ceil(num_subjs * test_percent))
num_train_subjs = num_subjs - num_test_subjs
subj_list = subj_dataset.dry_iter()
# Randomly shuffle the list of subjects, then choose the first `num_test_subjs` subjects
# for testing.
random.shuffle(subj_list)

# Create partial function to collect list of samples and form a tuple of tensors.
collate_fn = functools.partial(
    pitn.samplers.collate_subj, full_res_key="fr_dti", low_res_key="lr_dti"
)

test_dataset = torchio.SubjectsDataset(subj_list[:num_test_subjs], load_getitem=False)
# Choose the remaining for training/validation.
# If only 1 subject is available, assume this is a debugging run.
if num_subjs == 1 and num_train_subjs == 0:
    print("DEBUG: Only 1 subject with no training subjects, mixing train and test set")
    subj_list = subj_list[:]
    num_train_subjs = num_test_subjs
else:
    subj_list = subj_list[num_test_subjs:]

train_dataset = torchio.SubjectsDataset(subj_list, load_getitem=False)

# Training patch sampler, random across all patches of all volumes.
train_sampler = pitn.samplers.MultiresSampler(
    source_img_key="fr_dti",
    low_res_key="lr_dti",
    downsample_factor=downsample_factor,
    label_name="fr_brain_mask",
    source_spatial_patch_size=output_spatial_patch_shape,
    low_res_spatial_patch_size=input_spatial_patch_shape,
    label_probabilities={0: 0, 1: 1},
)

patches_per_subj = 8000
# Hold enough for 2 epochs at a time.
queue_max_length = patches_per_subj * num_train_subjs * 2

# Set up a torchio.Queue to act as a sampler proxy for the torch DataLoader
train_queue = torchio.Queue(
    train_dataset,
    max_length=queue_max_length,
    samples_per_volume=patches_per_subj,
    sampler=train_sampler,
    shuffle_patches=True,
    shuffle_subjects=True,
    num_workers=6,
    #     verbose=True,
)

train_loader = torch.utils.data.DataLoader(
    train_queue,
    batch_size=batch_size,
    collate_fn=collate_fn,
    pin_memory=True,
    num_workers=0,
)

# Set up testing objects.
# Calculate the patch overlap needed for ~50% of the patch volume overlapping (which is
# not the same as dividing each dimension by 2).
input_vol_half_overlap = int(
    np.floor(np.power(np.prod(input_spatial_patch_shape) / 2, 1 / 3))
)
# torchio requires an even-numbered overlap.
if input_vol_half_overlap % 2 == 1:
    input_vol_half_overlap += 1
input_vol_half_overlap = np.repeat(input_vol_half_overlap, 3)

# Repeat for the output.
output_vol_half_overlap = np.floor(
    np.power(np.prod(output_spatial_patch_shape) / 2, 1 / 3)
).astype(int)
# torchio requires an even-numbered overlap.
if output_vol_half_overlap % 2 == 1:
    output_vol_half_overlap += 1
output_vol_half_overlap = np.repeat(output_vol_half_overlap, 3)

# Test samplers
test_samplers = list()
for subj in test_dataset.dry_iter():
    test_samplers.append(
        pitn.samplers.MultiresGridSampler(
            subject=subj,
            source_img_key="fr_dti",
            low_res_key="lr_dti",
            downsample_factor=downsample_factor,
            source_spatial_patch_size=output_spatial_patch_shape,
            low_res_spatial_patch_size=input_spatial_patch_shape,
            patch_overlap=output_vol_half_overlap,
            # Due to the oversampling in the LR->HR patch mapping, a mask is necessary
            # to avoid sampling from out of bounds.
            source_mask=subj["fr_brain_mask"].tensor[0].bool(),
        )
    )

concat_test_dataset = torch.utils.data.ConcatDataset(test_samplers)
test_loader = torch.utils.data.DataLoader(
    concat_test_dataset,
    batch_size=batch_size,
    collate_fn=collate_fn,
    pin_memory=True,
    num_workers=6,
)


print("Test subject(s) IDs: ", [s.subj_id for s in test_dataset.dry_iter()])
print("Training subject(s) IDs: ", [s.subj_id for s in train_dataset.dry_iter()])

In [ ]:
with open(experiment_results_log, "a+") as f:
    f.write(f"Training Set Subjects: {[s.subj_id for s in test_dataset.dry_iter()]}\n")
    f.write(f"Test Set Subjects: {[s.subj_id for s in train_dataset.dry_iter()]}\n")

## Model Definition

In [ ]:
# Full pytorch-lightning module for contained training, validation, and testing.
debug_prob_threshold = 0.01
class DIQTSystem(pl.LightningModule):
    def __init__(
        self,
        channels,
        downsample_factor,
        train_loss_method: str,
        norm_method=None,
        train_loss_log_file=None,
    ):
        super().__init__()

        self._channels = channels
        self._downsample_factor = downsample_factor

        # Parameters
        # Network parameters
        self.net = pitn.nn.models.ThreeConv(
            self._channels, self._downsample_factor, norm_method=norm_method
        )

        ## Training parameters
        self._lr = 10e-3
        self._betas = (0.9, 0.999)
        loss_methods = {
            "MSE".casefold(): torch.nn.MSELoss(reduction="mean"),
            "SSE".casefold(): torch.nn.MSELoss(reduction="sum"),
            "RMSE".casefold(): lambda y_hat, y: torch.sqrt(
                F.mse_loss(y_hat, y, reduction="mean")
            ),
            "L1".casefold(): torch.nn.L1Loss(reduction="mean"),
        }

        try:
            self._loss_fn = loss_methods[train_loss_method.casefold()]
        except (AttributeError, KeyError) as e:
            if callable(train_loss_method):
                self._loss_fn = train_loss_method
            else:
                raise e

        # My own dinky logging object.
        self.plain_log = {"train_loss": list(), "val_loss": list(), "test_loss": list()}
        self.train_loss_log_file = train_loss_log_file
        if self.train_loss_log_file is not None:
            with open(self.train_loss_log_file, "a+") as f:
                f.write("epoch, batch_idx, loss\n")

    @staticmethod
    def normalize_with_layer(norm_layer, y):

        if norm_layer is not None:
            if not norm_layer.track_running_stats:
                y = norm_layer(y)
            else:
                if isinstance(norm_layer, torch.nn.InstanceNorm3d):
                    y = F.instance_norm(
                        y,
                        eps=norm_layer.eps,
                    )
                elif isinstance(norm_layer, torch.nn.BatchNorm3d):
                    y = F.batch_norm(
                        y,
                        eps=norm_layer.eps,
                    )

        return y

    def forward(self, x):
        y = self.net(x)
        return y

    def training_step(self, batch, batch_idx):
        x, y = batch

        # We need both prediction and ground truth to be a standard normal distribution
        # for a fair calculation of the loss.
        y_pred = self.net(x, norm_output=False)
        if self.net.norm is not None:
            y = self.normalize_with_layer(self.net.norm, y)

        loss = self._loss_fn(y_pred, y)
        if random.random() < debug_prob_threshold:
            breakpoint()
        self.log("train_loss", loss, logger=False)
        self.plain_log["train_loss"].append(float(loss.cpu()))
        return loss

    def training_epoch_end(self, training_step_outputs):
        # Only log each epoch if a log filename was given.
        if self.train_loss_log_file:

            row_iter = itertools.product(
                (self.current_epoch,),
                range(len(training_step_outputs)),
                [str(float(l["loss"].detach().cpu())) for l in training_step_outputs],
            )

            loss_rows = "".join(
                [
                    f"{epoch}, {batch_idx}, {loss}\n"
                    for epoch, batch_idx, loss in row_iter
                ],
            )

            with open(self.train_loss_log_file, "a+") as f:
                f.write(loss_rows)

    #     def validation_step(self, batch, batch_idx):
    #         pass

    def test_step(self, batch, batch_idx):

        x, y = batch

        # We can't normalize the outputs or ground truth because it would change
        # our units out of $mm^2/s$.
        y_pred = self.net(x, norm_output=False)
        #         if self.net.norm is not None:
        #             y = self.normalize_with_layer(self.net.norm, y)

        test_loss = torch.sqrt(F.mse_loss(y_pred, y, reduction="mean"))
        self.log("test_loss", test_loss)
        self.plain_log["test_loss"].append(float(test_loss.cpu()))

        return test_loss

    def viz_step(self, x, norm_output=True):
        """Step for running inference for the purpose of a visualization.

        Mainly deals with normalization."""

        with torch.no_grad():
            if self.net.training:
                was_training = True
            else:
                was_training = False

            self.net.eval()

            y_pred = self.net(x, norm_output=norm_output)

            if isinstance(self.net.norm, torch.nn.InstanceNorm3d):
                target_mean = torch.mean(x, dim=(2, 3, 4), keepdim=True)
                target_var = torch.var(x, dim=(2, 3, 4), keepdim=True)
                eps = self.net.norm.eps

            elif isinstance(self.net.norm, torch.nn.BatchNorm3d):
                target_mean = torch.mean(x, dim=(0, 2, 3, 4), keepdim=True)
                target_var = torch.var(x, dim=(0, 2, 3, 4), keepdim=True)
                eps = self.net.norm.eps
            else:
                target_mean = torch.zeros(1, 1, 1, 1, 1).to(x)
                target_var = torch.ones(1, 1, 1, 1, 1).to(x)
                eps = 1e-10

            y_pred = pitn.data.norm.denormalize_batch(
                y_pred, target_mean, target_var, eps=eps
            )

            if was_training:
                self.net.train()

        return y_pred

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(
            self.net.parameters(), lr=self._lr, betas=self._betas
        )
        return optimizer

## Training Loop

In [ ]:
train_loss_log_file = experiment_results_dir / "train_loss.csv"

train_start_timestamp = datetime.datetime.now().replace(microsecond=0)

model = DIQTSystem(
    channels=channels,
    downsample_factor=downsample_factor,
    train_loss_log_file=train_loss_log_file,
    norm_method=network_norm_method,
    train_loss_method=train_loss_name,
)

with open(experiment_results_log, "a+") as f:
    f.write(f"Max epochs: {max_epochs}\n")
    f.write(f"Model overview: {model}\n")
    f.write(f"Training loss function: {train_loss_name}\n")

# Create trainer object. Note: `automatic_optimization` needs to be set to `False` when
# manually performing backprop. See
# <https://colab.research.google.com/drive/1nGtvBFirIvtNQdppe2xBes6aJnZMjvl8?usp=sharing>
trainer = pl.Trainer(gpus=1, max_epochs=max_epochs, progress_bar_refresh_rate=10)

trainer.fit(model, train_loader)
train_duration = datetime.datetime.now().replace(microsecond=0) - train_start_timestamp
print(f"Train duration: {train_duration}")

In [ ]:
with open(experiment_results_log, "a+") as f:
    f.write("\n")
    f.write(f"Training time: {train_duration}\n")
    f.write(
        f"\t{train_duration.days} Days, "
        + f"{train_duration.seconds // 3600} Hours,"
        + f"{(train_duration.seconds // 60) % 60} Minutes,"
        + f'{train_duration.seconds % 60} Seconds"\n'
    )

In [ ]:
# Plot rolling average window of training loss values.
plt.figure(dpi=110)
window = 1000
rolling_mean = (
    np.convolve(model.plain_log["train_loss"], np.ones(window), "valid") / window
)
rolling_start = 100
plt.plot(
    np.arange(
        window + rolling_start,
        window + rolling_start + len(rolling_mean[rolling_start:]),
    ),
    rolling_mean[rolling_start:],
)
plt.title("Training Loss " + train_loss_name + f"\nRolling Mean {window}")
plt.xlabel("Epoch")
plt.ylabel("Loss")
# plt.ylim(0, 1)
print(np.median(rolling_mean))
print(
    np.mean(model.plain_log["train_loss"][: window + rolling_start]),
    np.var(model.plain_log["train_loss"][: window + rolling_start]),
    np.max(model.plain_log["train_loss"][: window + rolling_start]),
)

plt.savefig(experiment_results_dir / "train_loss.png")

# Model Testing/Evaluation

## Testing Loop

In [ ]:
trainer.test(test_dataloaders=test_loader)

In [ ]:
test_loss_name = "RMSE"
test_loss_log_file = experiment_results_dir / "test_loss.csv"

with open(experiment_results_log, "a+") as f:
    f.write(f"Test loss function: {test_loss_name}\n")

row_iter = enumerate(model.plain_log["test_loss"])
test_loss_tabular = "".join([f"{batch_idx}, {loss}\n" for batch_idx, loss in row_iter])

with open(test_loss_log_file, "a+") as f:
    f.write("batch_idx, loss\n")
    f.write(test_loss_tabular)

## Cubic Spline Baseline

In [ ]:
cubic_spline_test_log = list()

for subj in test_dataset.dry_iter():
    print("---")
    target_shape = subj["fr_dti"]["data"].cpu().numpy().shape
    interp_cubic_spline = scipy.ndimage.zoom(
        subj["lr_dti"]["data"].cpu().numpy(),
        zoom=(1, downsample_factor, downsample_factor, downsample_factor),
        order=spline_interp_order,
    )
    if interp_cubic_spline.shape != target_shape:
        # Crop off the end few voxels to account for the lack of padding used in full-
        # volume inference.
        interp_cubic_spline = interp_cubic_spline[
            :, : target_shape[1], : target_shape[2], : target_shape[3]
        ]
    print(f"Subj {subj['subj_id']} done")

    cubic_spline_test_log.append(interp_cubic_spline)

cspline_loss = list()

# Calculate spline loss for test images.
for subj, cspline_pred in zip(test_dataset.dry_iter(), cubic_spline_test_log):
    # De-normalize the ground truth volume.
    gt = subj["fr_dti"]["data"]
    gt = gt.detach().cpu().numpy()
    gt = (gt * (subj["fr_vars"] + 1 ** (-10))) + subj["fr_means"]
    # De-normalize the spline prediction to match the channel-wise distribution of the
    # ground truth.
    cspline_pred = (cspline_pred * (subj["fr_vars"] + 10 ** (-10))) + subj["fr_means"]
    # Calculate the RMSE of just the values found in the mask.
    se = (gt - cspline_pred) ** 2
    se = se[:, subj["fr_brain_mask"]["data"].bool().cpu().numpy()[0]]
    loss = np.sqrt(se.mean())
    cspline_loss.append(loss)

# Find the grand mean of the spline RMSE's
cspline_loss_mean = np.mean(cspline_loss)
print(cspline_loss_mean)

## Evaluation Visualization

In [ ]:
# Plot testing loss values over all patches.
fig, ax_prob = plt.subplots(figsize=(8, 4), dpi=120)
log_scale = True

sns.histplot(
    np.asarray(model.plain_log["test_loss"]),
    kde=True,
    stat="probability",
    log_scale=log_scale,
    ax=ax_prob,
    label="Current Model",
    legend=False,
)
plt.xlabel("Loss in $mm^2/second$")
# Draw an entire other plot, invisibly, to have another y-axis.
ax_count = ax_prob.twinx()
sns.histplot(
    np.asarray(model.plain_log["test_loss"]),
    alpha=0,
    ax=ax_count,
    stat="count",
    log_scale=log_scale,
)

# Draw means of different comparison models.
comparison_kwargs = {"ls": "--", "alpha": 0.8, "lw": 2.5}
# Plot the current DNN model performance.
plt.axvline(
    np.asarray(model.plain_log["test_loss"]).mean(),
    label="Current Model Mean",
    color="blue",
    **comparison_kwargs,
)
# Our spline mean performance.
plt.axvline(
    cspline_loss_mean,
    label="(Ours) C-spline Mean",
    color="black",
    **comparison_kwargs,
)
# Tanno, et. al., 2021 model comparisons.
# Taken from Table 2, HCP exterior.
plt.axvline(
    31.738 * (10.0 ** (-4)),
    label="(Tanno etal, 2021) C-spline Mean",
    color="red",
    **comparison_kwargs,
)
plt.axvline(
    23.139 * (10.0 ** (-4)),
    label="(Tanno etal, 2021) RF",
    color="orange",
    **comparison_kwargs,
)
plt.axvline(
    13.609 * (10.0 ** (-4)),
    label="(Tanno etal, 2021) ESPCN Baseline",
    color="green",
    **comparison_kwargs,
)
plt.axvline(
    13.412 * (10.0 ** (-4)),
    label="(Tanno etal, 2021) Best",
    color="purple",
    **comparison_kwargs,
)
# Best performing Blumberg, et. al., 2018 paper model.
plt.axvline(
    12.78 * (10.0 ** (-4)),
    label="(Blumberg etal, 2018) Best",
    color="pink",
    **comparison_kwargs,
)

plt.legend(fontsize="small")
plt.title(f"Test Loss Histogram with Test Metric {test_loss_name}")
plt.savefig(experiment_results_dir / "test_loss_hist.png")

In [ ]:
print(np.asarray(model.plain_log["test_loss"]).max())
print(np.asarray(model.plain_log["test_loss"]).mean())

# Whole-Volume Visualization

In [ ]:
# Create full 3D volume of full-res ground truth, low-res downsample, and high-res
# inferences.
@dataclass
class SubjResult:
    subj_id: int
    full_res: torch.Tensor
    low_res: torch.Tensor
    full_res_predicted: torch.Tensor
    full_res_cubic_spline: np.ndarray


test_vol_results = list()

with torch.no_grad():

    for subj in test_dataset.dry_iter():

        # Create a grid sampler for this subject.
        subj_sampler = pitn.samplers.MultiresGridSampler(
            subject=subj,
            source_img_key="fr_dti",
            low_res_key="lr_dti",
            downsample_factor=downsample_factor,
            source_spatial_patch_size=output_spatial_patch_shape,
            low_res_spatial_patch_size=input_spatial_patch_shape,
            source_mask=subj["fr_brain_mask"].tensor,
            patch_overlap=0,
        )

        loader = torch.utils.data.DataLoader(
            subj_sampler, batch_size=256, pin_memory=True
        )
        aggregator = torchio.GridAggregator(subj_sampler)

        # Iterate over all batches of patches.
        for batch in loader:

            x = batch["lr_dti"]["data"]
            # Locations are in reference to the full-res groud truth.
            locations = batch["location"]
            predictions = (
                model.viz_step(x.to(model.device), norm_output=False).detach().cpu()
            )

            aggregator.add_batch(predictions, locations)

        # Collect all variants of the volume and aggregate into one container object.

        # Calculate cubic spline as a baseline.
        full_res_cubic_spline = scipy.ndimage.zoom(
            subj["lr_dti"]["data"].cpu().numpy(),
            zoom=((1,) + (downsample_factor,) * 3),
            order=3,
        )
        full_res_cubic_spline = torch.from_numpy(full_res_cubic_spline).to(fr_vol)
        fr_vol = subj["fr_dti"]["data"]
        lr_vol = subj["lr_dti"]["data"]

        full_res_predicted = aggregator.get_output_tensor()

        if data_norm_method is not None and "channel" in data_norm_method.casefold():

            fr_means = torch.as_tensor(subj["fr_means"]).to(fr_vol)
            fr_vars = torch.as_tensor(subj["fr_vars"]).to(fr_vol)
            fr_vol = pitn.data.norm.denormalize_dti(fr_vol, fr_means, fr_vars)
            lr_means = torch.as_tensor(subj["lr_means"]).to(lr_vol)
            lr_vars = torch.as_tensor(subj["lr_vars"]).to(lr_vol)
            lr_vol = pitn.data.norm.denormalize_dti(lr_vol, lr_means, lr_vars)

            full_res_cubic_spline = pitn.data.norm.denormalize_dti(
                full_res_cubic_spline, fr_means, fr_vars
            )
            full_res_predicted = pitn.data.norm.denormalize_dti(
                full_res_predicted, fr_means, fr_vars
            )
        # Zero-out all voxels outside the mask.
        fr_mask = subj["fr_brain_mask"]["data"]
        full_res_cubic_spline = full_res_cubic_spline * fr_mask.to(
            full_res_cubic_spline
        )
        fr_vol = fr_vol * fr_mask.to(fr_vol)
        lr_vol = lr_vol * subj["lr_brain_mask"]["data"].to(lr_vol)

        subj_result = SubjResult(
            subj_id=subj["subj_id"],
            full_res=fr_vol,
            low_res=lr_vol,
            full_res_predicted=full_res_predicted,
            full_res_cubic_spline=full_res_cubic_spline,
        )

        test_vol_results.append(subj_result)

In [ ]:
vis_subj_idx = 0

In [ ]:
# Generate FA-weighted diffusion direction map for prediction.
tensor_key = "full_res_predicted"
pred_dir_map = pitn.viz.direction_map(
    test_vol_results[vis_subj_idx].__getattribute__(tensor_key).data.cpu().numpy()
)
# Set channels last for matplotlib
pred_dir_map = pred_dir_map.transpose(1, 2, 3, 0)

In [ ]:
# Generate FA-weighted diffusion direction map for cubic spline interpolation.
tensor_key = "full_res_cubic_spline"
cspline_dir_map = pitn.viz.direction_map(
    test_vol_results[vis_subj_idx].__getattribute__(tensor_key).data.cpu().numpy()
)
# Set channels last for matplotlib
cspline_dir_map = cspline_dir_map.transpose(1, 2, 3, 0)

In [ ]:
# Generate FA-weighted diffusion direction map.
tensor_key = "full_res"
fr_dir_map = pitn.viz.direction_map(
    test_vol_results[vis_subj_idx].__getattribute__(tensor_key).data.cpu().numpy()
)
# Set channels last for matplotlib
fr_dir_map = fr_dir_map.transpose(1, 2, 3, 0)

In [ ]:
# Generate FA-weighted diffusion direction map for low-res input.
tensor_key = "low_res"
lr_dir_map = pitn.viz.direction_map(
    test_vol_results[vis_subj_idx].__getattribute__(tensor_key).data.cpu().numpy()
)
# Set channels last for matplotlib
lr_dir_map = lr_dir_map.transpose(1, 2, 3, 0)

In [ ]:
slice_idx = (slice(None, None, None), slice(None, None, None), 100)
low_res_slice_idx = tuple(s // 2 if isinstance(s, int) else s for s in slice_idx)
print(slice_idx)
print(low_res_slice_idx)

In [ ]:
cspline_dir_map.shape
fr_dir_map.shape

In [ ]:
plt.figure(dpi=150)
plt.imshow(np.rot90(pred_dir_map[slice_idx]))
plt.axis("off")
plt.savefig(experiment_results_dir / "pred_dir_map_sample.png");

In [ ]:
plt.figure(dpi=150)
plt.imshow(np.rot90(cspline_dir_map[slice_idx]))
plt.axis("off")
plt.savefig(experiment_results_dir / "cubic_spline_dir_map_sample.png");

In [ ]:
plt.figure(dpi=150)
plt.imshow(np.rot90(fr_dir_map[slice_idx]))
# plt.colorbar()
plt.axis("off")
plt.savefig(experiment_results_dir / "ground_truth_dir_map_sample.png")

In [ ]:
plt.figure(dpi=150)
plt.imshow(np.rot90(lr_dir_map[low_res_slice_idx]))
plt.axis("off")
plt.savefig(experiment_results_dir / "low_res_map_sample.png");

## Per-Image Normalization

In [ ]:
# Display all 6 DTIs for ground truth, predicted, and root squared error

channel_names = ["Dxx", "Dxy", "Dyy", "Dxz", "Dyz", "Dzz"]
dti_names = [
    "Full-Res",
    "Low-Res Input",
    "Cubic Spline",
    "Predicted",
    "Root Squared Error\nFR vs. Prediction",
]
cmap = "jet"
# cmaps = ["Reds", "Greys", "Greens", "Purples", "Greys", "Blues", ]

dtis = [
    test_vol_results[vis_subj_idx]
    .full_res.data[(slice(None), *slice_idx)]
    .cpu()
    .numpy(),
    test_vol_results[vis_subj_idx]
    .low_res[(slice(None), *low_res_slice_idx)]
    .cpu()
    .numpy(),
    test_vol_results[vis_subj_idx].full_res_cubic_spline[(slice(None), *slice_idx)],
    test_vol_results[vis_subj_idx]
    .full_res_predicted[(slice(None), *slice_idx)]
    .cpu()
    .numpy(),
]

# Add root squared error between FR and prediction.
dtis.append(np.sqrt((dtis[0] - dtis[-1]) ** 2))

nrows = len(dtis)
ncols = len(channel_names)

fig = plt.figure(figsize=(12, 6), dpi=160)

grid = mpl.gridspec.GridSpec(
    nrows,
    ncols,
    figure=fig,
    hspace=0.05,
    wspace=0.05,
)

max_subplot_height = 0
for i_row in range(nrows):
    dti = dtis[i_row]

    for j_col in range(ncols):
        ax = fig.add_subplot(grid[i_row, j_col])
        ax.imshow(np.rot90(dti[j_col]), cmap=cmap, interpolation=None)
        if ax.get_subplotspec().is_first_col():
            ax.set_ylabel(dti_names[i_row], size="small")
        if ax.get_subplotspec().is_last_row():
            ax.set_xlabel(channel_names[j_col])

        # Update highest subplot to put the `suptitle` later on.
        max_subplot_height = max(
            max_subplot_height, ax.get_position(original=False).get_points()[1, 1]
        )
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_xticklabels([])
        ax.set_yticklabels([])
plt.suptitle(
    "DTI Channel Breakdown, Per-Image Normalization",
    y=max_subplot_height + 0.015,
    verticalalignment="bottom",
)
plt.savefig(experiment_results_dir / "DTI_channel_sample_per_img_norm.png");

## Global Normalization

In [ ]:
# Display all 6 DTIs for ground truth, predicted, and root squared error

dti_names = [
    "Full-Res",
    "Low-Res Input",
    "Cubic Spline",
    "Predicted",
    "Root Squared Error\nFR vs. Prediction",
]
cmap = "jet"
# cmaps = ["Reds", "Greys", "Greens", "Purples", "Greys", "Blues", ]

dtis = [
    test_vol_results[vis_subj_idx]
    .full_res.data[(slice(None), *slice_idx)]
    .cpu()
    .numpy(),
    test_vol_results[vis_subj_idx]
    .low_res[(slice(None), *low_res_slice_idx)]
    .cpu()
    .numpy(),
    test_vol_results[vis_subj_idx].full_res_cubic_spline[(slice(None), *slice_idx)],
    test_vol_results[vis_subj_idx]
    .full_res_predicted[(slice(None), *slice_idx)]
    .cpu()
    .numpy(),
]

# Add root squared error
dtis.append(np.sqrt((dtis[0] - dtis[-1]) ** 2))

# Don't take the absolute max and min values, as there exist some extreme (e.g., > 3
# orders of magnitude) outliers. Instead, take some percente quantile.
# Reshape and concatenate the dtis in order to compute the quantiles of images with
# different shapes (e.g., the low-res input patch).
max_dti = np.quantile(np.concatenate([di.reshape(6, -1) for di in dtis], axis=1), 0.95)
min_dti = np.quantile(np.concatenate([di.reshape(6, -1) for di in dtis], axis=1), 0.05)

nrows = len(dtis)
ncols = len(channel_names)

fig = plt.figure(figsize=(12, 6), dpi=160)

grid = mpl.gridspec.GridSpec(
    nrows,
    ncols,
    figure=fig,
    hspace=0.05,
    wspace=0.05,
)
axs = list()
max_subplot_height = 0
for i_row in range(nrows):
    dti = dtis[i_row]

    for j_col in range(ncols):
        ax = fig.add_subplot(grid[i_row, j_col])
        ax.imshow(
            np.rot90(dti[j_col]),
            cmap=cmap,
            interpolation=None,
            vmin=min_dti,
            vmax=max_dti,
        )
        if ax.get_subplotspec().is_first_col():
            ax.set_ylabel(dti_names[i_row], size="small")
        if ax.get_subplotspec().is_last_row():
            ax.set_xlabel(channel_names[j_col])
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        # Update highest subplot to put the `suptitle` later on.
        max_subplot_height = max(
            max_subplot_height, ax.get_position(original=False).get_points()[1, 1]
        )
        axs.append(ax)

color_norm = mpl.colors.Normalize(vmin=min_dti, vmax=max_dti)
fig.colorbar(
    mpl.cm.ScalarMappable(norm=color_norm, cmap=cmap),
    ax=axs,
    location="right",
    fraction=0.1,
    pad=0.03,
)
plt.suptitle(
    "DTI Channel Breakdown, Normalized over All Images",
    y=max_subplot_height + 0.015,
    verticalalignment="bottom",
)
plt.savefig(experiment_results_dir / "DTI_channel_sample_global_norm.png");

## Channel-Wise Normalization

In [ ]:
# Display all 6 DTIs for ground truth, predicted, low-res input, and root squared error
# Normalize by index in the DTI coefficients.
# Reshape and concatenate the dtis in order to compute the quantiles of images with
# different shapes (e.g., the low-res input patch).

channel_names = ["Dxx", "Dxy", "Dyy", "Dxz", "Dyz", "Dzz"]
dti_names = [
    "Full-Res",
    "Low-Res Input",
    "Cubic Spline",
    "Predicted",
    "Root Squared Error\nFR vs. Prediction",
]
cmap = "coolwarm"
# cmaps = ["Reds", "Greys", "Greens", "Purples", "Greys", "Blues", ]

dtis = [
    test_vol_results[vis_subj_idx]
    .full_res.data[(slice(None), *slice_idx)]
    .cpu()
    .numpy(),
    test_vol_results[vis_subj_idx]
    .low_res[(slice(None), *low_res_slice_idx)]
    .cpu()
    .numpy(),
    test_vol_results[vis_subj_idx].full_res_cubic_spline[(slice(None), *slice_idx)],
    test_vol_results[vis_subj_idx]
    .full_res_predicted[(slice(None), *slice_idx)]
    .cpu()
    .numpy(),
]

# Add root squared error
dtis.append(np.sqrt((dtis[0] - dtis[-1]) ** 2))


# Don't take the absolute max and min values, as there exist some extreme (e.g., > 3
# orders of magnitude) outliers. Instead, take some percente quantile.
max_dtis = np.quantile(
    np.concatenate([di.reshape(6, -1) for di in dtis], axis=1), 0.95, axis=1
)
min_dtis = np.quantile(
    np.concatenate([di.reshape(6, -1) for di in dtis], axis=1), 0.05, axis=1
)

max_dtis = np.max(np.abs([max_dtis, min_dtis]), axis=0)
min_dtis = -1 * max_dtis

nrows = len(dtis)
ncols = len(channel_names)

fig = plt.figure(figsize=(12, 7), dpi=160)

grid = mpl.gridspec.GridSpec(
    nrows,
    ncols,
    figure=fig,
    hspace=0.05,
    wspace=0.05,
)

axs = list()
max_subplot_height = 0
for i_row in range(nrows):
    dti = dtis[i_row]
    axs_cols = list()

    for j_col in range(ncols):
        ax = fig.add_subplot(grid[i_row, j_col])
        ax.imshow(
            np.rot90(dti[j_col]),
            cmap=cmap,
            interpolation=None,
            vmin=min_dtis[j_col],
            vmax=max_dtis[j_col],
        )
        if ax.get_subplotspec().is_first_col():
            ax.set_ylabel(dti_names[i_row], size="small")
        if ax.get_subplotspec().is_last_row():
            ax.set_xlabel(channel_names[j_col])

        # Update highest subplot to put the `suptitle` later on.
        max_subplot_height = max(
            max_subplot_height, ax.get_position(original=False).get_points()[1, 1]
        )
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_xticklabels([])
        ax.set_yticklabels([])

        axs_cols.append(ax)

    axs.append(axs_cols)

# Place colorbars on each column.
for j_col in range(ncols):

    full_col_ax = [axs[i][j_col] for i in range(nrows)]

    color_norm = mpl.colors.Normalize(vmin=min_dtis[j_col], vmax=max_dtis[j_col])

    color_mappable = mpl.cm.ScalarMappable(norm=color_norm, cmap=cmap)
    cbar = fig.colorbar(
        color_mappable,
        ax=full_col_ax,
        location="top",
        orientation="horizontal",
        pad=0.01,
        shrink=0.85,
    )
    cbar.ax.tick_params(labelsize=8, rotation=35)
    cbar.ax.xaxis.set_major_formatter(mpl.ticker.StrMethodFormatter("{x:g}"))
#     cbar.ax.ticklabel_format(scilimits=(3, -3), useOffset=False)

plt.suptitle(
    "DTI Channel Breakdown, Channel-Wise Normalization",
    y=max_subplot_height + 0.01,
    verticalalignment="bottom",
)
plt.savefig(experiment_results_dir / "DTI_channel_sample_channel_wise_norm.png");

In [ ]:
# Experiment is complete, move the results directory to its final location.
if experiment_results_dir != final_experiment_results_dir:
    print("Moving out of tmp location")
    experiment_results_dir = experiment_results_dir.rename(final_experiment_results_dir)